In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import normal, seed
from scipy.odr import *

seed(42)

sx = 1e-5
yslope = 1e-6 + 1
sy = sx*1e3*yslope
x = np.arange(1, 5, 1) + normal(0, sx, 4)
print(x)
y = x*yslope + normal(0, sy, 4)
print(y)

[1.00000497 1.99999862 3.00000648 4.00001523]
[0.99766443 1.99765925 3.01580162 4.00769359]


In [12]:
def odrfit(x, y, sx, sy):

    def linfun(B, x):
        '''Linear function y = m*x + b'''
        return B[0]*x + B[1]

    linear = Model(linfun)

    mydata = Data(x, y, wd=1./sx**2, we=1./sy**2)

    myodr = ODR(mydata, linear, beta0=[1., 2.])

    myoutput = myodr.run()

    #myoutput.pprint()
    slope=myoutput.beta[0]
    offset=myoutput.beta[1]
    ressquared = np.sum((y - linfun(myoutput.beta, x))**2)
    #ressquared = myoutput.sum_square
    return slope, offset, ressquared
    
def linearregression(x, y, sx, sy):
    
    sy = np.asarray([sy]*len(x))

    def linfun(B, x):
        '''Linear function y = m*x + b'''
        return B[0]*x + B[1]

    popt = np.polyfit(x, y, 1, w=1/sy)

    #myoutput.pprint()
    slope=popt[0]
    offset=popt[1]
    ressquared = np.sum((y - np.poly1d(popt)(x))**2)
    return slope, offset, ressquared

fitmethod = odrfit
#fitmethod = linearregression
    
slope, offset, res = fitmethod(x, y, sx, sy)
print(slope, offset, res)

1.0048190998914046 -0.007349381955367829 0.00011497518820976799


In [13]:
scale = 1e-4
slope2, offset2, res2 = fitmethod(x*scale, y, sx*scale, sy)
print(slope2*scale, offset2)
print(slope2*scale/slope-1, offset2/offset-1, res2/res-1)

1.0048191039983805 -0.00734939488099539
4.087278959730156e-09 1.7587366720750452e-06 9.15489906105904e-13


In [14]:
offset = 1e-4
slope3, offset3, res3 = fitmethod(x+offset, y, sx, sy)
print(slope3, offset3)
print(slope3/slope-1, offset3/offset-1, res3/res-1)

1.0048190996943542 -0.007449865662625843
-1.9610535417768915e-10 -75.49865662625842 8.548717289613705e-14


In [15]:
offset = 1e-4
scale = 1e-6
slope4, offset4, res4 = fitmethod(x*scale+offset, y, sx*scale, sy)
print(slope4, offset4)
print(slope4*scale/slope-1, offset4/offset-1, res4/res-1)

1004819.1002811944 -100.48925941155878
3.8792036249901685e-10 -1004893.5941155878 -2.7755575615628914e-15


In [16]:
thescale = np.asarray([1/(1+42/2000/A) for A in [168, 170, 172, 174]])
print(thescale)
print(thescale/thescale[0]-1)
slope5, offset5, res5 = fitmethod(x*thescale, y, sx, sy)
print(slope5, offset5)
print(slope5/slope-1, offset5/offset-1, res5/res-1)

[0.99987502 0.99987649 0.99987792 0.99987932]
[0.00000000e+00 1.47040660e-06 2.90662187e-06 4.30982468e-06]
1.0049389198317185 -0.007342187022354896
0.00011924528537199386 -74.42187022354895 0.0001941662938718114


In [17]:

slope6, offset6, res6 = fitmethod(x*thescale, y*thescale, sx, sy)
print(slope6, offset6)
print(slope6/slope-1, offset6/offset-1, res6/res-1)

1.0048190827070382 -0.007348436011819448
-1.7101950411380074e-08 -74.48436011819447 -0.0002442547470367096
